In [ ]:
import scipy
# scipy?

import numpy as np
np.random
import scipy.special as spec

In [ ]:
import sympy as sp
sp.init_printing()

In [ ]:
x, y, xi = sp.symbols('x y xi')
z = sp.sqrt(x**2 + y**2)
z

In [ ]:
(z).diff(x).integrate(x)

In [ ]:
hbar, E, m, x = sp.symbols('hbar, E, m, x', real = True, positive = True)
psi = sp.Function('psi')(x)
psi

schrodinger = sp.Eq(E * psi, -hbar**2/2/m * psi.diff(x,2))
schrodinger

In [ ]:

rozwiazanie = sp.dsolve(schrodinger, psi).rhs.subs({hbar: 1, m: 1, 'C1': 1, 'C2': 0.3, E:1})
# rozwiazanie
# for e in range(1, 2):
#     sp.plot(rozwiazanie.subs({E: e}))

In [ ]:
funkcja_schrodinger = sp.lambdify(x, rozwiazanie, 'mimpy')
import numpy as np
x_plot = np.linspace(-10, 10, 100)
y_plot = funkcja_schrodinger(x_plot)
import matplotlib.pyplot as plt
plt.plot(x_plot, y_plot)

In [ ]:
plt.show()plt.show()

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.random((200,5)),
                  columns=["pierwsza", "druga", "trzecia", "bęc", "dzieńdobry"])
df.T #transpozycja
df.cumsum()
df['bęc']
df2 = pd.DataFrame([[1, 2, 3], [2, 17, 42]]) #np.loadtxt("pani_aniu_phodaje_dane.txt")

#problem: wszystkie komórki z kolumny df['bęc'] tam gdzie średnia wartość w rzędzie jest < 0.5
pd.DataFrame(df[df.T.mean() < 0.5]['bęc'])
df[df>0.3]

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ani

In [16]:
k = 1
J = 1
T = 2.26

N = 16   #NxN spinów w siatce
NT = 100000 #liczba kroków czasowych
snap_every = 1000

N_snapshots = NT // snap_every
beta = 1/k/T

spins = np.random.randint(0,2, size = (N, N))*2 - 1
magnetyzacja = spins.sum()

def plot(spins):
    fig = plt.figure()
    axis = fig.add_subplot(111)
    axis.imshow(spins, cmap='Greys', interpolation='none', origin='lower')
    axis.set_title(T)
    plt.show()
    
def flip_spin(spins):
    """
    1. wybieramy losowo spin
        * losujemy x, y = random int[0, N)
    2. liczymy energię
        * E = - J spin[x,y] * suma po sąsiadach z uwzględnieniem OWB (PBC)
    3. liczymy prawdopodobieństwo przerzucenia = exp(-2*E*beta)
    4. losujemy uniform(0, 1)
    5. jeśli boltzmann > uniform(0,1): przerzucamy
    6. losowo wybrany spin jest przerzucony do góry nogami z prawdopodobieństwem bosmanowskim
    """
    global magnetyzacja
    
    x, y = np.random.randint(0, N, size=2)
    E = 2 * J * spins[x, y] * (spins[(x+1)%N, y] + spins[(x-1)%N, y] +
                           spins[x, (y+1)%N] + spins[x, (y-1)%N])
    boltzmann = np.exp(-E*beta)
    uniform = np.random.random()
    if uniform < boltzmann:
        zmiana_magnetyzacji = -2 * spins[x,y]
        magnetyzacja += zmiana_magnetyzacji
        spins[x,y] = spins[x,y] * -1

spins_history = np.empty((N_snapshots, N, N))
mag_history = np.empty(NT)
for i in range(NT):
    if(i % snap_every == 0):
        spins_history[i//snap_every] = spins
    mag_history[i] = magnetyzacja
    
    flip_spin(spins)
def plot_mag():
    plt.plot(mag_history/N**2)
    plt.show()
    
    
def animacja(): #nie musi brać argumentu bo niczego nie modyfikuje
    fig = plt.figure()
    axis = fig.add_subplot(111)
    #zmiana 1 poniżej: IM =
                     #spins_history[0] zamiast spins
    IM = axis.imshow(spins_history[0], cmap='Greys', interpolation='none', origin='lower')
    # zmiana 2
    title = axis.set_title("T = {}, i = {}".format(T, 0))
    
    def animate(i_snapshot):
        IM.set_array(spins_history[i_snapshot])
        title.set_text("T = {}, i = {}".format(T, i_snapshot*snap_every))
        return [IM, title]
    
    animacja_obiekt = ani.FuncAnimation(fig, animate, frames = N_snapshots)
    plt.show()
animacja()
plot_mag()

In [47]:
import scipy.fftpack as fft
# plot_mag()

N_punktow = 10000
t = np.arange(N_punktow)
okres = 1000
y = np.sin(2*np.pi*t/okres) + np.sin(2*np.pi*t/(okres/10))

f = fft.rfftfreq(N_punktow)
y_fft = fft.rfft(y)**2
y_fft[0] = 5e8
plt.plot(f, y_fft, "bo-")
plt.show()

y_odwrotna = np.pi*fft.irfft(y_fft)/N_punktow
plt.plot(t, y)
plt.plot(t, y_odwrotna, "r-")
plt.show()



In [55]:
#poisson 1d przez fouriera
N = 1000
x = np.linspace(-10, 10, N)
dx = x[1] - x[0]
# dywergencja z potencjału = gęstość ładunku / epsilon_0 (1)
# d2phi/dx2 = rho
# po sfourierzeniu
# d/dx -> -i*k
# (d/dx)^2 -> -k^2
# -k^2 phi = rho
# phi = -rho/k^2

epsilon_0 = 1
rho = np.sin(x**2)
k = fft.rfftfreq(N, dx)
phi_f = -fft.rfft(rho)/k**2
phi_f[0] = 0
phi = fft.irfft(phi_f)/N #*normalizacja?????

plt.plot(x, rho)
plt.plot(x, phi)
plt.show()

/home/dominik/.anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: RuntimeWarning: divide by zero encountered in true_divide


In [66]:
import numba
A = np.arange(100000)**2
B = np.arange(100000)**3
def dodawanie(A, B):
    return A + B

In [67]:
%%timeit
dodawanie(A,B)

1000 loops, best of 3: 1.07 ms per loop


In [68]:
@numba.jit
def dodawanie_numba(A, B):
    return A + B

In [69]:
%%timeit
dodawanie_numba(A,B)

The slowest run took 1090.86 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 1.01 ms per loop
